In [28]:
import numpy as np
from napari_stress import vectors
import vedo
from scipy.interpolate import RegularGridInterpolator
import napari
import pandas as pd

In [29]:
image = np.random.rand(100,100,100)
sphere = vedo.Sphere(r=10, pos=(50,50,50))

In [30]:
vectors = vectors.normal_vectors_on_pointcloud(sphere.points())

100%|██████████| 1/1 [00:00<00:00, 166.71it/s]


In [31]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_points(sphere.points(), size=0.5, face_color='red')
viewer.add_vectors(vectors, edge_color='blue')


10-Feb-23 15:02:17 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 2172x1352+0+20 (frame: 2198x1423-13-38) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 4350x2119+5+47 (frame: 4376x2190-8-11) margins: 13, 58, 13, 13 minimum size: 374x560 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=774,1191 maxtrack=0,0)


<Vectors layer 'vectors' at 0x27f03d50a90>

In [32]:
# Create coords for interpolator
X1 = np.arange(0, image.shape[0], 1)
X2 = np.arange(0, image.shape[1], 1)
X3 = np.arange(0, image.shape[2], 1)

In [33]:
interpolator = RegularGridInterpolator((X1, X2, X3),
                                        image,
                                        bounds_error=False,
                                        fill_value=np.nan)

In [34]:
start_points = vectors[:, 0]

In [35]:
unit_vector = vectors[:, 1]/np.linalg.norm(vectors[:, 1], axis=1)[:, np.newaxis]

In [36]:
sampling_distance = 0.5
steps = np.round(np.linalg.norm(vectors[:, 1], axis=1)[:, np.newaxis] / sampling_distance).mean().astype(int)

intensity = np.zeros((len(start_points), steps))
for step in range(steps):
    sampling_coordinates = np.stack([start_points[idx] + step * sampling_distance * unit_vector[idx] for idx in range(len(start_points))])
    intensity[:, step] = interpolator(sampling_coordinates)

In [37]:
intensity = pd.DataFrame(intensity)
intensity.columns = [f'step_{idx}' for idx in range(steps)]

In [38]:
intensity

,step_0,step_1
0,0.854492,0.903508
1,0.912932,0.472480
2,0.459012,0.687322
3,0.413512,0.536467
4,0.102012,0.255283
...,...,...
1053,0.398117,0.556531
1054,0.594177,0.704409
1055,0.546623,0.660726
1056,0.491679,0.609282
